In [1]:
import numpy as np
import pandas as pd

In [6]:
def utility(mode, Lambda, dataset):
    '''
    Get the utility for mode j under specific OD pair
    Lambda: parameter that trade-off different transportation mode
    '''
    subset = dataset[dataset['mode'] == mode]
    wagelist = ['2500', '7500', '12500', '17500', '22500', '30000', '42500', '62500', '87500', '125000', '225000']
    for index, row in subset.iterrows():
        time = row['duration']
        price = row['price']
        vj = 0
        for wage in wagelist:
            wagePercent = row[wage]
            vj += float(wagePercent)*(Lambda*(float(time)*(float(wage)/525600) + float(price))) #525600: convert wage scale 'year' to 'minitues'
    return -vj #assign negative utility?


def InclusiveValue(Nk, Tk, Lambda, dataset):
    '''
    Get the inclusive value for nest K
    Nk:nest k
    T: the dict that contains Tao(dissmilarity parameter) for each Nest. for example, Tk meeas Tao for nest K
    '''
    subsetNk = dataset[dataset['nest'] == Nk]
    modes = list(subsetNk['mode'].unique()) #what modes contained in this nest k
    sumIV = 0
    for j in modes:
        vj = utility(j, Lambda, subsetNk)
        dictVj[j] = vj
        sumIV += np.exp((1/Tk)*vj)  
    IVk = np.log(sumIV)
    return dictVj, IVk
    

def denoSum(T, nestList, Lambda, dataset):
    '''
    Calculate the denomenator for P(y=Nk)
    T: the dict that contains Tk for each Nest, in our case T={1:T1, 2:T2, 3:T3}; T1, T2, T3 defined by us 
    TotK: the total number of nest this model has, in our case TotK = 3
    '''
    deno = 0
    dictIVk = {}
    for Nk in nestList: #k is the k Nest, in our case k=1,2,3
        Tk = T[Nk] #get the tao for nest k 
        dictVj, IVk = InclusiveValue(Nk, Tk, Lambda, dataset)
        denok = np.exp(Tk*IVk)
        deno += denok
        dictIVk[Nk] = IVk
    return dictVj, dictIVk, deno


def probability (j, Nk, T, dictVj, dictIVk, deno):  
    '''
    Calculate the probability for the specific mode j and Nest Nk
    '''
    Tk = T[Nk] #Tk is the Tao for specific Nest K
    vj = dictVj[j]
    IVk = dictIVk[Nk]
    pjk = np.exp((1/Tk)*vj)/np.exp(IVk) #pjk: P(y=j, y belong to Nk)
    pk = np.exp(Tk*IVk)/deno #pk: P(y belong to Nk)
    pj = pjk*pk #pj: P(y=j)
    return pj


## Apply our dataset: Transportaton proportion based on OD pair

In [7]:
dataAll = pd.read_csv('final_allMode_with_wage.csv').set_index('ODpair')
dataAll['duration'] = dataAll['duration'].map(lambda x: x/60) #convert the scale of duration from seconds to minutes
dataAll.head(10)

,Unnamed: 0,12500,125000,17500,22500,225000,2500,30000,42500,62500,7500,87500,DOlocationID,PUlocationID,duration,mode,nest,price
ODpair,,,,,,,,,,,,,,,,,,
3-1,0,0.108075,0.123454,0.047035,0.069126,0.085667,0.041803,0.098668,0.11188,0.156432,0.060036,0.097823,1,3,56.650000,1,1,100.000000
3-1,1,0.108075,0.123454,0.047035,0.069126,0.085667,0.041803,0.098668,0.11188,0.156432,0.060036,0.097823,1,3,55.922881,2,1,126.000000
3-1,2,0.108075,0.123454,0.047035,0.069126,0.085667,0.041803,0.098668,0.11188,0.156432,0.060036,0.097823,1,3,62.420000,3,1,120.000000
3-1,3,0.108075,0.123454,0.047035,0.069126,0.085667,0.041803,0.098668,0.11188,0.156432,0.060036,0.097823,1,3,128.000000,4,2,5.500000
3-1,4,0.108075,0.123454,0.047035,0.069126,0.085667,0.041803,0.098668,0.11188,0.156432,0.060036,0.097823,1,3,569.083333,5,3,0.000000
3-2,5,0.108075,0.123454,0.047035,0.069126,0.085667,0.041803,0.098668,0.11188,0.156432,0.060036,0.097823,2,3,140.000000,4,2,8.250000
3-2,6,0.108075,0.123454,0.047035,0.069126,0.085667,0.041803,0.098668,0.11188,0.156432,0.060036,0.097823,2,3,353.800000,5,3,0.000000
3-3,7,0.108075,0.123454,0.047035,0.069126,0.085667,0.041803,0.098668,0.11188,0.156432,0.060036,0.097823,3,3,6.508943,1,1,16.647073
3-4,8,0.108075,0.123454,0.047035,0.069126,0.085667,0.041803,0.098668,0.11188,0.156432,0.060036,0.097823,4,3,39.695000,2,1,64.000000


In [8]:
#Prepare the new dataset "OD_modeProportion" to store the proportion predicted by our model
#'-' will be replaced by the predicted probability of each mode after run the model
OD_pair = dataAll.index.unique()
OD_modeProportion = pd.DataFrame(dataAll.index.unique(), columns=['ODpair']).set_index('ODpair')
OD_modeProportion['P(mode1)'] = '-'
OD_modeProportion['P(mode2)'] = '-'
OD_modeProportion['P(mode3)'] = '-'
OD_modeProportion['P(mode4)'] = '-'
OD_modeProportion['P(mode5)'] = '-'
OD_modeProportion.head()

,P(mode1),P(mode2),P(mode3),P(mode4),P(mode5)
ODpair,,,,,
3-1,-,-,-,-,-
3-2,-,-,-,-,-
3-3,-,-,-,-,-
3-4,-,-,-,-,-
3-5,-,-,-,-,-


In [9]:
#Use OD pair '3-1' to Test Function
T1 = 0.3 #set grid search for T1 
T2 = 1
T3 = 1
T = {1:T1, 2:T2, 3:T3} # T={1: T1, 2:T2, 3:T3}; T1, T2, T3 defined by us 
Lambda = 1 #set grid search for Lambda 
totK = 3
modeList = [1, 2, 3, 4, 5]

dataZone = dataAll[dataAll.index=='3-1']
nestList = dataZone['nest'].unique()
dictVj = {}
dictVj, dictIVk, deno = denoSum(T, nestList, Lambda, dataZone)
try:
    p_mode1nest1 = probability(1, 1, T, dictVj, dictIVk, deno)
except:
    p_mode1nest1 = 'Nan'
try:
    p_mode2nest1 = probability(2, 1, T, dictVj, dictIVk, deno)
except:
    p_mode2nest1 = 'Nan'
try:
    p_mode3nest1 = probability(3, 1, T, dictVj, dictIVk, deno)
except:
    p_mode3nest1 = 'Nan'
try:
    p_mode4nest2 = probability(4, 2, T, dictVj, dictIVk, deno)
except:
    p_mode4nest2 = 'Nan'
try:
    p_mode5nest3 = probability(5, 3, T, dictVj, dictIVk, deno)
except:
    p_mode5nest3 = 'Nan'
dictVj, dictIVk, deno, p_mode1nest1, p_mode2nest1, p_mode3nest1, p_mode4nest2, p_mode5nest3

({1: -107.01038825152911,
  2: -132.92040795144365,
  3: -127.72442073540066,
  4: -21.33988872366683,
  5: -70.42356776947967},
 {1: -356.7012941717637, 2: -21.33988872366683, 3: -70.42356776947967},
 5.397642079543515e-10,
 6.219776700785853e-38,
 1.9283218437119064e-75,
 6.414184393755254e-68,
 1.0,
 4.822020350315956e-22)

In [10]:
#Apply to all OD pairs

T1 = 5 #set grid search for T1 
T2 = 1
T3 = 1
T = {1:T1, 2:T2, 3:T3} # T={1: T1, 2:T2, 3:T3}; T1, T2, T3 defined by us 
Lambda = 2 #set grid search for Lambda 
nestList = dataZone['nest'].unique()

for i in OD_pair: #filter sub-dataset based on each OD-paired taxizone
    dataZone = dataAll[dataAll.index==i] #dataAll is the dataset that contains all modes 
    dictVj, dictIVk, deno = denoSum(T, nestList, Lambda, dataZone)
    
    try:
        p_mode1nest1 = probability(1, 1, T, dictVj, dictIVk, deno)
    except:
        p_mode1nest1 = 'Nan'
    try:
        p_mode2nest1 = probability(2, 1, T, dictVj, dictIVk, deno)
    except:
        p_mode2nest1 = 'Nan'
    try:
        p_mode3nest1 = probability(3, 1, T, dictVj, dictIVk, deno)
    except:
        p_mode3nest1 = 'Nan'
    try:
        p_mode4nest2 = probability(4, 2, T, dictVj, dictIVk, deno)
    except:
        p_mode4nest2 = 'Nan'
    try:
        p_mode5nest3 = probability(5, 3, T, dictVj, dictIVk, deno)
    except:
        p_mode5nest3 = 'Nan'


    #Assign the predicted proportion to the TPproportion datafrome
    OD_modeProportion.loc[i]['P(mode1)'] = p_mode1nest1
    OD_modeProportion.loc[i]['P(mode2)'] = p_mode2nest1
    OD_modeProportion.loc[i]['P(mode3)'] = p_mode3nest1
    OD_modeProportion.loc[i]['P(mode4)'] = p_mode4nest2
    OD_modeProportion.loc[i]['P(mode5)'] = p_mode5nest3


/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:31: RuntimeWarning: divide by zero encountered in log
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:59: RuntimeWarning: divide by zero encountered in double_scalars
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:61: RuntimeWarning: invalid value encountered in double_scalars


In [11]:
OD_modeProportion.head()

,P(mode1),P(mode2),P(mode3),P(mode4),P(mode5)
ODpair,,,,,
3-1,3.87295e-75,1.22183e-79,9.76442e-79,1,2.32519e-43
3-2,NaN,NaN,NaN,1,1.53148e-16
3-3,1,8.7333e-21,6.97933e-20,NaN,NaN
3-4,5.36132e-36,6.16327e-45,1.27469e-44,1,2.59266e-11
3-5,NaN,NaN,NaN,1,7.51796e-34


## Compared with ACS ground truth

#### convert above dataset by grouping by pickup taxizone:

In [143]:
taxi_zone_modeProportion = OD_modeProportion.copy()
for col in taxi_zone_modeProportion.columns:
    taxi_zone_modeProportion[col] = taxi_zone_modeProportion[col].astype(float)
taxi_zone_modeProportion['taxi_zone'] = OD_modeProportion.index.map(lambda x: x.split('-')[0])
#taxi_zone_modeProportion = taxi_zone_modeProportion.fillna(0)
taxi_zone_modeProportion = taxi_zone_modeProportion.groupby('taxi_zone').mean().reset_index()
taxi_zone_modeProportion['taxi_zone'] = taxi_zone_modeProportion['taxi_zone'].astype(int)
taxi_zone_modeProportion.head()

,taxi_zone,P(mode1),P(mode2),P(mode3),P(mode4),P(mode5)
0,10,0.020983,2.971921e+10,4.726177e+10,0.639761,0.330905
1,100,0.020163,1.156883e+05,3.972773e+05,0.899264,0.080033
2,101,0.017049,1.815878e-03,1.647057e-03,0.674136,0.311576
3,102,0.027550,5.357682e-03,5.350042e-03,0.660797,0.301950
4,103,0.000000,0.000000e+00,0.000000e+00,0.018413,0.981587


In [144]:
taxi_zone_modeProportion.dtypes

taxi_zone      int64
P(mode1)     float64
P(mode2)     float64
P(mode3)     float64
P(mode4)     float64
P(mode5)     float64
dtype: object

#### Groud truth proprotion:

In [145]:
acs = pd.read_csv('ACS_transportation_choice.csv')
acs = acs.rename(columns={'FHV proportion':'P(mode1)',
                         'FHV_shared proportion':'P(mode2)',
                         'Taxi proportion':'P(mode3)',
                         'pt':'P(mode4)',
                         'walked':'P(mode5)'})
# acs = acs.set_index('taxi_zone')
acs.drop('Unnamed: 0', axis=1, inplace=True)
acs.head()

,taxi_zone,P(mode1),P(mode2),P(mode3),P(mode4),P(mode5)
0,3,0.009394,0.003862,0.000061,0.880923,0.105760
1,4,0.010074,0.006050,0.003064,0.666539,0.314272
2,5,0.005482,0.000883,0.000034,0.977816,0.015785
3,6,0.006416,0.001578,0.000092,0.894894,0.097021
4,7,0.004083,0.001076,0.000731,0.914596,0.079514


In [146]:
acs.dtypes

taxi_zone      int64
P(mode1)     float64
P(mode2)     float64
P(mode3)     float64
P(mode4)     float64
P(mode5)     float64
dtype: object

In [160]:
data_compare = pd.merge(taxi_zone_modeProportion, acs, left_on='taxi_zone', right_on = 'taxi_zone', how = 'left')
data_compare = data_compare.dropna()
print(data_compare.shape)
data_compare.head()

(241, 11)


,taxi_zone,P(mode1)_x,P(mode2)_x,P(mode3)_x,P(mode4)_x,P(mode5)_x,P(mode1)_y,P(mode2)_y,P(mode3)_y,P(mode4)_y,P(mode5)_y
0,10,0.020983,2.971921e+10,4.726177e+10,0.639761,0.330905,0.010678,0.004535,0.000830,0.923040,0.060917
1,100,0.020163,1.156883e+05,3.972773e+05,0.899264,0.080033,0.019578,0.003700,0.068047,0.488584,0.420091
2,101,0.017049,1.815878e-03,1.647057e-03,0.674136,0.311576,0.018926,0.002999,0.000317,0.865214,0.112544
3,102,0.027550,5.357682e-03,5.350042e-03,0.660797,0.301950,0.003454,0.001158,0.000018,0.883582,0.111789
5,106,341429.836448,3.788796e+04,1.539318e+05,0.953616,0.019085,0.001062,0.000193,0.000052,0.903984,0.094709


In [161]:
col = data_compare.columns
col

Index(['taxi_zone', 'P(mode1)_x', 'P(mode2)_x', 'P(mode3)_x', 'P(mode4)_x',
       'P(mode5)_x', 'P(mode1)_y', 'P(mode2)_y', 'P(mode3)_y', 'P(mode4)_y',
       'P(mode5)_y'],
      dtype='object')

In [162]:
def dfmse(df, col):
    mse = 0
    for i in range(1,6):
        mse += (df[col[i]] - df[col[i+5]])**2
    return sum(mse)

In [163]:
mse = dfmse(data_compare, col)
mse

4.723840422380772e+112